In [ ]:
import os
os.chdir('../src')

os.environ["CUDA_VISIBLE_DEVICES"]="0"

### load getData module

In [ ]:
%run modules/data/getData.py

In [ ]:
data_shape    = (56, 56, 3)

### Get properly formatted input data --> (image_id, image_height, image_width, channel)

In [ ]:
inputDataDict = getInputDataDict( resize_shape=data_shape )

In [ ]:
print( inputDataDict['train_images'].shape, inputDataDict['train_images'].max(), inputDataDict['train_images'].min() )
print( inputDataDict['test_images'].shape, inputDataDict['test_images'].max(), inputDataDict['test_images'].min() )

In [ ]:
trainData   = inputDataDict['train_images'].copy()
np.random.shuffle( trainData )
trainData   = trainData[:100]
trainData.shape

In [ ]:
dataToVisualise = trainData[:, :, :, 0]
dataToVisualise = np.swapaxes( dataToVisualise, 0, 1 )
dataToVisualise = np.swapaxes( dataToVisualise, 1, 2 )
dataToVisualise.shape

In [ ]:
visualiseStackedArray(dataToVisualise)

### load getPretrained module

In [ ]:
%run modules/model/getPretrained.py

### Import autoencoder lib

In [ ]:
from lib.model import autoencoder

In [ ]:
ModelFileDict = getModelFileDict()
ModelFileDict

In [ ]:
model = modelImageNet('VGG16', ModelFileDict['VGG16'], input_shape=data_shape)

In [ ]:
model.summary()

In [ ]:
preTrainedLayers = model.layers
preTrainedLayers

In [ ]:
inputData = preTrainedLayers[0].output
inputData

In [ ]:
maxPool_lastLayer = preTrainedLayers[-1]
print(maxPool_lastLayer)
print(maxPool_lastLayer.output)

In [ ]:
output_lastLayer  = maxPool_lastLayer.output

In [ ]:
with tf.variable_scope('Flatten'):
    flattenLayer  = tf.layers.Flatten(data_format='channels_last', name='flattenLayer')
    flattenOutput = flattenLayer(output_lastLayer)

In [ ]:
flattenOutput

In [ ]:
aeNetwork         = autoencoder.AE_simple(layers=[300, 200], 
                                          activations=[tf.tanh, tf.tanh],
                                          latentDim=10)

In [ ]:
with tf.variable_scope('Autoencoder_Network'):
    finalOutput         = aeNetwork( flattenOutput )
    reconstructionLayer = tf.layers.Dense(units=np.prod(data_shape), activation=tf.nn.sigmoid, name='reconstructionLayer')
    finalOutput         = reconstructionLayer( finalOutput )
    finalOutput         = tf.reshape(finalOutput, shape=(-1, ) + data_shape, name='reshapeTo2D')

In [ ]:
finalOutput

In [ ]:
with tf.variable_scope('Optimisation'):
    err                 = tf.losses.mean_squared_error(inputData, finalOutput)
    opt                 = tf.train.AdamOptimizer().minimize( err )

In [ ]:
outputTensorBoard('VGG16_autoencoder')

Before we start training, let's turn off "trainable" for all pre-trained layers

In [ ]:
for l in preTrainedLayers:
    l.trainable = False

In [ ]:
modelNew = keras.models.Model(inputs=model.input, outputs=reconstructionLayer.output)

In [ ]:
modelNew.summary()

In [ ]:
tf_config = tf.ConfigProto(gpu_options={'allow_growth' : True,
                                        'per_process_gpu_memory_fraction' : 0.3
                                        },
                           allow_soft_placement = True,
                           log_device_placement = False)

with tf.Session(config=tf_config) as sess:
    
    sess.run( tf.global_variables_initializer() )
    pbar  = tqdm( range(1000) )
    
    for i in pbar:
        _, err_out = sess.run( [opt, err], 
                               feed_dict={
                                   'input_1:0' : trainData
                               })
        description = 'err_out: {:.3f}'.format( err_out )
        pbar.set_description( description )
        
    constructedImgs = sess.run( finalOutput,
                                feed_dict={
                                    'input_1:0' : trainData
                                })

In [ ]:
constructedImgs = constructedImgs[:, :, :, 0]
constructedImgs = np.swapaxes(constructedImgs, 0, 1)
constructedImgs = np.swapaxes(constructedImgs, 1, 2)
constructedImgs.shape

In [ ]:
visualiseStackedArray(constructedImgs)